# Calculating model climo of impacts
### National, State, CWA

- Create index of days through the year [0...366]
- For each day, we have {Natl: {min:, 10:, 50:, 90:, max:}, {States: {AL: {min:, 10:, 50:, 90:, max:}}, {cwas: {ALY: {min:, 10:, 50:, 90:, max:}}}
- Open each sim file, grab day and calculate stats for nat, states, and cwas
- Add the quantiles to the running total for each
- Divide by the number of total years for each day to acquire the average of each stat 
- Run a 7-day running average on the stats

In [8]:
import datetime
import glob
import json
import numpy as np
import pandas as pd
import math

In [25]:
files = glob.glob('/Volumes/Backup Plus/pas_output/simulationFiles/*')

# Takes the file suffix and grabs the date index
#datetime.datetime.strptime(files[-180].split('/')[-1].split('_')[0][:-4],'%Y%m%d').strftime('%j')

Grab state and cwa lists

In [12]:
with open('../visualization/exampleDashboard/includes/jsons/states.json') as json_file: 
    states = json.load(json_file)

with open('../visualization/exampleDashboard/includes/jsons/cwa.json') as json_file:
    cwas = json.load(json_file)

Set up large dictionary to hold quantile sums

In [14]:
def makeStatHolder():
    
    with open('../visualization/exampleDashboard/includes/jsons/states.json') as json_file: 
        states = json.load(json_file)

    with open('../visualization/exampleDashboard/includes/jsons/cwa.json') as json_file:
        cwas = json.load(json_file)

    statHolder = {'nat': {'min':np.zeros(366),'ten':np.zeros(366),'med':np.zeros(366),'ninety':np.zeros(366),'max':np.zeros(366)},'states': {}, 'cwas': {}}

    for state in states:
        statHolder['states'][state['abbreviation']] = {'min':np.zeros(366),'ten':np.zeros(366),'med':np.zeros(366),'ninety':np.zeros(366),'max':np.zeros(366)}

    for cwa in cwas:
        statHolder['cwas'][cwa['abbreviation']] = {'min':np.zeros(366),'ten':np.zeros(366),'med':np.zeros(366),'ninety':np.zeros(366),'max':np.zeros(366)}
        
        #print(cwa['abbreviation'])
    return statHolder

This is the main loop to run our climo

In [106]:
#statHolder = {'nat': {'min':np.zeros(366),'ten':np.zeros(366),'med':np.zeros(366),'ninety':np.zeros(366),'max':np.zeros(366)},'states': {}, 'cwas': {}}

statHolder = makeStatHolder()

impact = 'psubstations'

for i,file in enumerate(files):
    
    if (i%100 == 0):
        print(f'{i} file processing')
    
    # Get the index of the day and then subtract 1 for 0-indexing
    date = datetime.datetime.strptime(file.split('/')[-1].split('_')[0][:-4],'%Y%m%d')
    dayIdx = int(date.strftime('%j'))-1
    # If not a leap year and the dayIdx is beyond what would be the leap day, add 1 to the dayIdx
    if ((date.year%4 != 0) and (dayIdx > 58)):
        dayIdx += 1
    
    # Set up national stats
    df = pd.read_csv(file, sep="|")
    sims = df.groupby("sim")
    fields = sims.sum().loc[:,('population','hospitals','mobilehomes','psubstations')]
    # Fill missing sims with 0s
    fill_fields = fields.reindex(list(range(1,10001)),fill_value=0)
    
    # Get national quantiles for this file
    quants = fields.describe(percentiles=[0.1,0.5,0.9]).loc[('min','10%','50%','90%','max'),[impact]][impact].values.tolist()
    
    # Add quantiles to the proper slot in the national sub-dictionary
    statHolder['nat']['min'][int(dayIdx)] += quants[0]
    statHolder['nat']['ten'][int(dayIdx)] += quants[1]
    statHolder['nat']['med'][int(dayIdx)] += quants[2]
    statHolder['nat']['ninety'][int(dayIdx)] += quants[3]
    statHolder['nat']['max'][int(dayIdx)] += quants[4]
    
    
    ##### States #####
    
    stBrokenOut = df.assign(category=df['states'].str.split(',')).explode('category').reset_index(drop=True)

    # Remove row if state is NaN
    stBrokenOut = stBrokenOut[stBrokenOut['category'].notna()]

    # Grab a list of the unique states in the simulation
    statesImpacted = stBrokenOut['category'].unique().tolist()
    
    # Loop through impacted states
    for state in statesImpacted:

        # Organize dataframe for state and fill no-impact sims with 0s
        ind_state = stBrokenOut[stBrokenOut['category'] == state]
        #print(f'State: {state}')
        ind_state_group = ind_state.groupby("sim").sum().loc[:,['population','hospitals','mobilehomes','psubstations']]
        ind_state_tot = ind_state_group.reindex(list(range(1,10001)),fill_value=0)
        
        # Get state quantiles for this file
        quants = ind_state_tot.describe(percentiles=[0.1,0.5,0.9]).loc[['min','10%','50%','90%','max'],[impact]][impact].values.tolist()
        
        # Add quantiles to the proper slot in the national sub-dictionary
        statHolder['states'][state]['min'][int(dayIdx)] += quants[0]
        statHolder['states'][state]['ten'][int(dayIdx)] += quants[1]
        statHolder['states'][state]['med'][int(dayIdx)] += quants[2]
        statHolder['states'][state]['ninety'][int(dayIdx)] += quants[3]
        statHolder['states'][state]['max'][int(dayIdx)] += quants[4]
        
    ##### CWAs #####
    
    cwaBrokenOut = df.assign(category=df['wfos'].str.split(',')).explode('category').reset_index(drop=True)

    # Remove row if state is NaN
    cwaBrokenOut = cwaBrokenOut[cwaBrokenOut['category'].notna()]

    # Grab a list of the unique states in the simulation
    cwasImpacted = cwaBrokenOut['category'].unique().tolist()
    
    # Loop through impacted states
    for cwa in cwasImpacted:

        # Organize dataframe for state and fill no-impact sims with 0s
        ind_cwa = cwaBrokenOut[cwaBrokenOut['category'] == cwa]
        #print(f'State: {state}')
        ind_cwa_group = ind_cwa.groupby("sim").sum().loc[:,['population','hospitals','mobilehomes','psubstations']]
        ind_cwa_tot = ind_cwa_group.reindex(list(range(1,10001)),fill_value=0)
        
        # Get state quantiles for this file
        quants = ind_cwa_tot.describe(percentiles=[0.1,0.5,0.9]).loc[['min','10%','50%','90%','max'],[impact]][impact].values.tolist()
        
        try:
            # Add quantiles to the proper slot in the national sub-dictionary
            statHolder['cwas'][cwa]['min'][int(dayIdx)] += quants[0]
            statHolder['cwas'][cwa]['ten'][int(dayIdx)] += quants[1]
            statHolder['cwas'][cwa]['med'][int(dayIdx)] += quants[2]
            statHolder['cwas'][cwa]['ninety'][int(dayIdx)] += quants[3]
            statHolder['cwas'][cwa]['max'][int(dayIdx)] += quants[4]
        except:
            print(file)
    

0 file processing
100 file processing
200 file processing
300 file processing
400 file processing
500 file processing
600 file processing
700 file processing
800 file processing
900 file processing
1000 file processing
1100 file processing
1200 file processing
1300 file processing
1400 file processing
1500 file processing
1600 file processing
1700 file processing
1800 file processing
1900 file processing
2000 file processing
2100 file processing
2200 file processing
2300 file processing
2400 file processing
2500 file processing
2600 file processing
2700 file processing
2800 file processing
2900 file processing
3000 file processing
3100 file processing
3200 file processing
3300 file processing
3400 file processing
3500 file processing
3600 file processing
3700 file processing
3800 file processing
3900 file processing
4000 file processing
4100 file processing
4200 file processing
4300 file processing
4400 file processing
4500 file processing
4600 file processing
4700 file processing
4800

Divide the np arrays by the number of years for each position
Normal: 68
Leap: 17
Divide everything by 68, but then multiply the 59th index (leap day) by 4.

In [72]:
# A function to return the moving average from the input array and the moving window size
def movingaverage(array,window_size):
    
    subset_size = math.floor(window_size/2)
    new_arr = np.insert(array,0,array[-subset_size:])
    to_process = np.append(new_arr,array[:subset_size])
    
    window = np.ones(int(window_size))/float(window_size)
    return np.convolve(to_process,window,'valid')

In [25]:
# A dictionary to hold the raw averages
averages = makeStatHolder()
smoothed_averages = makeStatHolder()

In [107]:
# Create dictionaries to hold averages
averages = makeStatHolder()
smoothed_averages = makeStatHolder()

# Constants
tot_years = 68
leap_years = 17
window_size = 31

for i,key in enumerate(statHolder.keys()):
    if i == 0:
        quantKeys = statHolder[key].keys()
        for innerKey in quantKeys:
            # Create averages
            averages[key][innerKey] = statHolder[key][innerKey]/tot_years
            
            # Correct leap year average by multiplying by 4
            averages[key][innerKey][59] = averages[key][innerKey][59]*(tot_years/leap_years)
            
            # Create smoothed averages
            smoothed_averages[key][innerKey] = movingaverage(averages[key][innerKey],window_size)
            
    elif i == 1:
        states = statHolder[key].keys()
        for state in states:
            quantKeys = statHolder[key][state].keys()
            for innerKey in quantKeys:
                #statHolder[key][state][innerKey] = statHolder[key][state][innerKey].tolist()
                # Create averages
                averages[key][state][innerKey] = statHolder[key][state][innerKey]/tot_years

                # Correct leap year average by multiplying by 4
                averages[key][state][innerKey][59] = averages[key][state][innerKey][59]*(tot_years/leap_years)
                
                # Create smoothed averages
                smoothed_averages[key][state][innerKey] = movingaverage(averages[key][state][innerKey],window_size)
                
    else:
        cwas = statHolder[key].keys()
        for cwa in cwas:
            quantKeys = statHolder[key][cwa].keys()
            for innerKey in quantKeys:
                #statHolder[key][cwa][innerKey] = statHolder[key][cwa][innerKey].tolist()
                
                # Create averages
                averages[key][cwa][innerKey] = statHolder[key][cwa][innerKey]/tot_years

                # Correct leap year average by multiplying by 4
                averages[key][cwa][innerKey][59] = averages[key][cwa][innerKey][59]*(tot_years/leap_years)
                
                # Create smoothed averages
                smoothed_averages[key][cwa][innerKey] = movingaverage(averages[key][cwa][innerKey],window_size)

In [57]:
test_arr = np.array([1,2,3,4,5,6,7,8,9,19])
window_size = 7

def movingaverage(array,window_size):
    
    subset_size = math.floor(window_size/2)
    new_arr = np.insert(array,0,array[-subset_size:])
    to_process = np.append(new_arr,array[:subset_size])
    
    window = np.ones(int(window_size))/float(window_size)
    return np.convolve(to_process,window,'valid')
    
#np.convolve(test_arr,np.ones(int(window_size))/float(window_size),'valid')

In [110]:
smoothed_averages['nat']['med']

array([0.2272296 , 0.23055028, 0.2272296 , 0.22912713, 0.23766603,
       0.25711575, 0.27371917, 0.2528463 , 0.25616698, 0.24952562,
       0.24762808, 0.2414611 , 0.23956357, 0.25237192, 0.25      ,
       0.24573055, 0.24383302, 0.23197343, 0.22343454, 0.22248577,
       0.24240987, 0.24098672, 0.23197343, 0.22865275, 0.24240987,
       0.23529412, 0.23766603, 0.24051233, 0.23719165, 0.23719165,
       0.24620493, 0.25332068, 0.26470588, 0.278463  , 0.27656546,
       0.28605313, 0.2841556 , 0.27087287, 0.2841556 , 0.30597723,
       0.30597723, 0.31688805, 0.3240038 , 0.35673624, 0.36859583,
       0.39611006, 0.43121442, 0.44117647, 0.45683112, 0.47580645,
       0.50237192, 0.49288425, 0.50996205, 0.51185958, 0.53368121,
       0.5370019 , 0.5683112 , 0.59535104, 0.60009488, 0.61100569,
       0.62855787, 0.62049336, 0.63614801, 0.63187856, 0.65607211,
       0.67077799, 0.66461101, 0.65464896, 0.66413662, 0.66318786,
       0.65654649, 0.6902277 , 0.7243833 , 0.7414611 , 0.73339

In [48]:
new_arr = np.insert(test_arr,0,test_arr[-3:])
to_process = np.append(new_arr,test_arr[:3])
movingaverage(to_process,7)

array([6.57142857, 6.14285714, 5.71428571, 4.        , 5.        ,
       6.        , 8.28571429, 7.85714286, 7.42857143, 7.        ])

Convert ndarrays to lists so they can be serialized/saved

In [111]:
for i,key in enumerate(statHolder.keys()):
    if i == 0:
        quantKeys = statHolder[key].keys()
        for innerKey in quantKeys:
            statHolder[key][innerKey] = statHolder[key][innerKey].tolist()
            averages[key][innerKey] = averages[key][innerKey].tolist()
            smoothed_averages[key][innerKey] = smoothed_averages[key][innerKey].tolist()
            
    elif i == 1:
        states = statHolder[key].keys()
        for state in states:
            quantKeys = statHolder[key][state].keys()
            for innerKey in quantKeys:
                statHolder[key][state][innerKey] = statHolder[key][state][innerKey].tolist()
                averages[key][state][innerKey] = averages[key][state][innerKey].tolist()
                smoothed_averages[key][state][innerKey] = smoothed_averages[key][state][innerKey].tolist()
                
    else:
        cwas = statHolder[key].keys()
        for cwa in cwas:
            quantKeys = statHolder[key][cwa].keys()
            for innerKey in quantKeys:
                statHolder[key][cwa][innerKey] = statHolder[key][cwa][innerKey].tolist()
                averages[key][cwa][innerKey] = averages[key][cwa][innerKey].tolist()
                smoothed_averages[key][cwa][innerKey] = smoothed_averages[key][cwa][innerKey].tolist()
    
    #print(statHolder[key].keys())

array([ 2432., 19248., 16352., 10064., 37104.])

In [112]:
#type(statHolder['nat']['ten'])

with open('pow_climo_raw.json','w') as outfile:
    json.dump(statHolder,outfile)
    
with open('pow_climo_avg.json','w') as outfile2:
    json.dump(averages,outfile2)
    
with open('pow_climo_smAvg.json','w') as outfile3:
    json.dump(smoothed_averages,outfile3)

In [6]:
with open(f'../visualization/dashboard/backend/climo-data/pop_climo_raw.json') as file:
    pop_data = json.load(file)

In [39]:
def moving_average(a, n=3) :
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

### Get the # of tornado days (by dayIdx) for nat, each state, each CWA

In [17]:
def makeTorDayHolder():
    
    with open('../visualization/exampleDashboard/includes/jsons/states.json') as json_file: 
        states = json.load(json_file)

    with open('../visualization/exampleDashboard/includes/jsons/cwa.json') as json_file:
        cwas = json.load(json_file)

    statHolder = {'nat': np.zeros(366),'states': {}, 'cwas': {}}

    for state in states:
        statHolder['states'][state['abbreviation']] = np.zeros(366)

    for cwa in cwas:
        statHolder['cwas'][cwa['abbreviation']] = np.zeros(366)
        
        #print(cwa['abbreviation'])
    return statHolder

In [21]:
torDays = makeTorDayHolder()

In [26]:
torDays = makeTorDayHolder()

#impact = 'psubstations'

for i,file in enumerate(files):
    
    if (i%100 == 0):
        print(f'{i} file processing')
    
    # Get the index of the day and then subtract 1 for 0-indexing
    date = datetime.datetime.strptime(file.split('/')[-1].split('_')[0][:-4],'%Y%m%d')
    dayIdx = int(date.strftime('%j'))-1
    # If not a leap year and the dayIdx is beyond what would be the leap day, add 1 to the dayIdx
    if ((date.year%4 != 0) and (dayIdx > 58)):
        dayIdx += 1
    
    # Set up national stats
    df = pd.read_csv(file, sep="|")
    '''sims = df.groupby("sim")
    fields = sims.sum().loc[:,('population','hospitals','mobilehomes','psubstations')]
    # Fill missing sims with 0s
    fill_fields = fields.reindex(list(range(1,10001)),fill_value=0)
    
    # Get national quantiles for this file
    quants = fields.describe(percentiles=[0.1,0.5,0.9]).loc[('min','10%','50%','90%','max'),[impact]][impact].values.tolist()
    
    # Add quantiles to the proper slot in the national sub-dictionary
    statHolder['nat']['min'][int(dayIdx)] += quants[0]
    statHolder['nat']['ten'][int(dayIdx)] += quants[1]
    statHolder['nat']['med'][int(dayIdx)] += quants[2]
    statHolder['nat']['ninety'][int(dayIdx)] += quants[3]
    statHolder['nat']['max'][int(dayIdx)] += quants[4] '''
    
    
    torDays['nat'][int(dayIdx)] += 1
    
    
    ##### States #####
    
    stBrokenOut = df.assign(category=df['states'].str.split(',')).explode('category').reset_index(drop=True)

    # Remove row if state is NaN
    stBrokenOut = stBrokenOut[stBrokenOut['category'].notna()]

    # Grab a list of the unique states in the simulation
    statesImpacted = stBrokenOut['category'].unique().tolist()
    
    # Loop through impacted states
    for state in statesImpacted:
        
        torDays['states'][state][int(dayIdx)] += 1

        # Organize dataframe for state and fill no-impact sims with 0s
        '''ind_state = stBrokenOut[stBrokenOut['category'] == state]
        #print(f'State: {state}')
        ind_state_group = ind_state.groupby("sim").sum().loc[:,['population','hospitals','mobilehomes','psubstations']]
        ind_state_tot = ind_state_group.reindex(list(range(1,10001)),fill_value=0)
        
        # Get state quantiles for this file
        quants = ind_state_tot.describe(percentiles=[0.1,0.5,0.9]).loc[['min','10%','50%','90%','max'],[impact]][impact].values.tolist()
        
        # Add quantiles to the proper slot in the national sub-dictionary
        statHolder['states'][state]['min'][int(dayIdx)] += quants[0]
        statHolder['states'][state]['ten'][int(dayIdx)] += quants[1]
        statHolder['states'][state]['med'][int(dayIdx)] += quants[2]
        statHolder['states'][state]['ninety'][int(dayIdx)] += quants[3]
        statHolder['states'][state]['max'][int(dayIdx)] += quants[4]'''
        
    ##### CWAs #####
    
    cwaBrokenOut = df.assign(category=df['wfos'].str.split(',')).explode('category').reset_index(drop=True)

    # Remove row if state is NaN
    cwaBrokenOut = cwaBrokenOut[cwaBrokenOut['category'].notna()]

    # Grab a list of the unique states in the simulation
    cwasImpacted = cwaBrokenOut['category'].unique().tolist()
    
    # Loop through impacted states
    for cwa in cwasImpacted:
        
        torDays['cwas'][cwa][int(dayIdx)] += 1

        # Organize dataframe for state and fill no-impact sims with 0s
        '''ind_cwa = cwaBrokenOut[cwaBrokenOut['category'] == cwa]
        #print(f'State: {state}')
        ind_cwa_group = ind_cwa.groupby("sim").sum().loc[:,['population','hospitals','mobilehomes','psubstations']]
        ind_cwa_tot = ind_cwa_group.reindex(list(range(1,10001)),fill_value=0)
        
        # Get state quantiles for this file
        quants = ind_cwa_tot.describe(percentiles=[0.1,0.5,0.9]).loc[['min','10%','50%','90%','max'],[impact]][impact].values.tolist()
        
        try:
            # Add quantiles to the proper slot in the national sub-dictionary
            statHolder['cwas'][cwa]['min'][int(dayIdx)] += quants[0]
            statHolder['cwas'][cwa]['ten'][int(dayIdx)] += quants[1]
            statHolder['cwas'][cwa]['med'][int(dayIdx)] += quants[2]
            statHolder['cwas'][cwa]['ninety'][int(dayIdx)] += quants[3]
            statHolder['cwas'][cwa]['max'][int(dayIdx)] += quants[4]
        except:
            print(file)'''

0 file processing
100 file processing
200 file processing
300 file processing
400 file processing
500 file processing
600 file processing
700 file processing
800 file processing
900 file processing
1000 file processing
1100 file processing
1200 file processing
1300 file processing
1400 file processing
1500 file processing
1600 file processing
1700 file processing
1800 file processing
1900 file processing
2000 file processing
2100 file processing
2200 file processing
2300 file processing
2400 file processing
2500 file processing
2600 file processing
2700 file processing
2800 file processing
2900 file processing
3000 file processing
3100 file processing
3200 file processing
3300 file processing
3400 file processing
3500 file processing
3600 file processing
3700 file processing
3800 file processing
3900 file processing
4000 file processing
4100 file processing
4200 file processing
4300 file processing
4400 file processing
4500 file processing
4600 file processing
4700 file processing
4800

### Convert np arrays of tor days to lists for saving

In [42]:
for i,key in enumerate(torDays.keys()):
    if i == 0:
        torDays['nat'] = torDays['nat'].tolist()
    elif i == 1:
        states = torDays['states'].keys()
        for state in states:
            torDays['states'][state] = torDays['states'][state].tolist()
    elif i == 2:
        cwas = torDays['cwas'].keys()
        for cwa in cwas:
            torDays['cwas'][cwa] = torDays['cwas'][cwa].tolist()


In [43]:
with open('tor_days.json','w') as outfile:
    json.dump(torDays,outfile)

In [46]:
with open('tor_days.json','r') as tors:
    torlist = json.load(tors)

### Create moving averages using only tor days

Read in lists

In [157]:
with open('../climoFiles/hosp_climo_raw.json','r') as hosp:
    hosplist = json.load(hosp)

with open('../climoFiles/tor_days.json','r') as tor:
    torlist = json.load(tor)
    
with open('../climoFiles/mob_climo_raw.json','r') as mob:
    moblist = json.load(mob)
    
with open('../climoFiles/pow_climo_raw.json','r') as pow:
    powlist = json.load(pow)
    
with open('../climoFiles/pop_climo_raw.json','r') as pop:
    poplist = json.load(pop)

In [70]:
import math


In [71]:
window = 31
halfwin = math.floor(window/2)

rando = torlist['nat']
rando = np.array(rando)
rando1 = np.insert(rando,0,rando[-halfwin:])
rando2 = np.append(rando1,rando[:halfwin])

In [94]:
# This function adds values to the beginning and end to make it ready for summing
def arrExtender(array,window):
    halfwin = math.floor(window/2)
    
    rando = array
    rando = np.array(rando)
    rando1 = np.insert(rando,0,rando[-halfwin:])
    rando2 = np.append(rando1,rando[:halfwin])
    
    return rando2

In [109]:
def summer(array,window):

    total = np.array([])

    start = 0
    end = window
    while end < (len(array)+1):
        total = np.append(total,np.sum(array[start:end]))
        start += 1
        end += 1
        
    return total

In [112]:
# This is the 31-day running sum (centered on the middle date)
summer(arrExtender(hosplist['nat']['max'],window),window)/summer(arrExtender(torlist['nat'],window),window)

array([11.93706294, 11.95818815, 11.89690722, 11.9       , 11.9966443 ,
       12.15511551, 12.45394737, 12.2640264 , 12.29666667, 12.29470199,
       12.27574751, 12.21594684, 12.34437086, 12.3496732 , 12.26885246,
       12.24503311, 12.10223642, 11.8369906 , 11.75157233, 11.73650794,
       11.90822785, 12.        , 12.03236246, 12.05177994, 12.27741935,
       12.17088608, 12.2866242 , 12.36624204, 12.37539432, 12.38679245,
       12.30091185, 12.35311573, 12.38529412, 12.41107872, 12.38616715,
       12.46685879, 12.36723164, 12.19832402, 12.21823204, 12.33150685,
       12.40108401, 12.48773842, 12.45576408, 12.84168865, 12.82841823,
       12.8381201 , 12.92620865, 13.05012531, 13.19506173, 13.23021583,
       13.33018868, 13.08863636, 13.08849558, 13.09230769, 13.16198704,
       12.94092827, 13.02057613, 12.95219124, 12.80813953, 12.71374046,
       12.9738806 , 12.96296296, 12.96402878, 12.81643357, 12.99136442,
       12.9862543 , 12.87945671, 12.84317032, 12.79900332, 12.78

In [162]:
torDayStats = makeStatHolder()
impactList = poplist

for i,key in enumerate(torDayStats.keys()):
    if i == 0:
        
        torDays = summer(arrExtender(torlist['nat'],window),window)
        
        quantKeys = torDayStats[key].keys()
        for innerKey in quantKeys:
            # Create list of daily 31-day averages
            torDayStats[key][innerKey] = (summer(arrExtender(impactList[key][innerKey],window),window)/torDays).tolist()
         
    elif i == 1:
        
        states = torDayStats[key].keys()
        for state in states:
            
            # Get the tor days for individual states
            torDays = summer(arrExtender(torlist['states'][state],window),window)
        
        #quantKeys = torDayStats[key].keys()
            for innerKey in quantKeys:
                # Create list of daily 31-day averages
                torDayStats[key][state][innerKey] = (summer(arrExtender(impactList[key][state][innerKey],window),window)/torDays).tolist()

              
    else:
        
        cwas = torDayStats[key].keys()
        for cwa in cwas:
            
            # Get the tor days for individual CWAs
            torDays = summer(arrExtender(torlist['cwas'][cwa],window),window)
        
        #quantKeys = torDayStats[key].keys()
            for innerKey in quantKeys:
                # Create list of daily 31-day averages
                torDayStats[key][cwa][innerKey] = (summer(arrExtender(impactList[key][cwa][innerKey],window),window)/torDays).tolist()

/Users/josephpicca/anaconda/envs/impacts/lib/python3.7/site-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in true_divide
/Users/josephpicca/anaconda/envs/impacts/lib/python3.7/site-packages/ipykernel_launcher.py:39: RuntimeWarning: invalid value encountered in true_divide


In [163]:
with open('../climoFiles/pop_climo_torSmAvg.json','w') as outfile:
    json.dump(torDayStats,outfile)

In [170]:
with open('../climoFiles/pop_climo_torSmAvg.json','r') as outfile:
    test = json.load(outfile)

In [172]:
test['nat']['min'][140]

98.39481437831468